## Training a model for duration prediction - Yellow trip dataset

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

We'll use [the same NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

### Q1. Read the data for January. 

**How many columns are there?**

In [1]:
!pip install pyarrow

In [2]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer

In [3]:
df = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
print(df.shape)
print(len(df.columns))

(3066766, 19)
19


Answer: 19 columns

### Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

**What's the standard deviation of the trips duration in January?**

In [5]:
df['trip_duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.trip_duration = df.trip_duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
df.trip_duration.describe(percentiles=[0.95,0.98,0.99])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
50%      1.151667e+01
95%      3.646667e+01
98%      4.873333e+01
99%      5.725000e+01
max      1.002918e+04
Name: trip_duration, dtype: float64

Answer: The standard deviation is: 42.59

### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

**What fraction of the records left after you dropped the outliers?**

In [7]:
df_without_outliers = df[(df.trip_duration >= 1) & (df.trip_duration <= 60)]

In [8]:
records_after_dropout = len(df_without_outliers) / len(df)
records_after_dropout

0.9812202822125979

Answer: 98%

### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
<!--  -->
Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it

**What's the dimensionality of this matrix (number of columns)?**

In [9]:
categorical = ['PULocationID', 'DOLocationID']

df_filtered = df.copy()
df_filtered[categorical] = df_filtered[categorical].astype(str)

train_dicts = df_filtered[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(len(dv.feature_names_))

518


Answer: 515 

### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data

**What's the RMSE on train?**

In [10]:
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

In [11]:
target = 'trip_duration'
y_train = df_filtered[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

41.996492307453806

Answer: 7.64

### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [12]:
# free memory in Codespaces
del df
del df_filtered
del train_dicts
del X_train
del y_train

import gc
gc.collect()

0

In [13]:
df = pd.read_parquet("data/yellow_tripdata_2023-02.parquet")
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [17]:

df['trip_duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.trip_duration = df.trip_duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.trip_duration >= 1) & (df.trip_duration <= 60)]

In [18]:
categorical = ['PULocationID', 'DOLocationID']

df = df.copy()
df[categorical] = df[categorical].astype(str)

val_dicts = df[categorical].to_dict(orient='records')

In [19]:

X_val = dv.transform(val_dicts)  

y_val = df['trip_duration'].values
y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.945263293094538